<a href="https://colab.research.google.com/github/hjesse92/style_transfer_w266/blob/main/Few_Shot_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Few Shot Learning

## Setup

In [2]:
!pip install -q transformers datasets sentencepiece rouge_score accelerate evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.2 MB/s eta 0:0

In [3]:
#Am I running a GPU and what type is it?
!nvidia-smi

Sun Mar 19 04:35:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    17W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import torch

# Clear out cuda
torch.cuda.empty_cache()

if torch.cuda.is_available():     
    device = torch.device("cuda")
    print('Number of GPU(s) available:', torch.cuda.device_count())
    print('GPU device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available')
    device = torch.device("cpu")

Number of GPU(s) available: 1
GPU device name: Tesla T4


In [22]:
from logging import warning
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler, TensorDataset

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, AdamW, Seq2SeqTrainingArguments, Seq2SeqTrainer


from sklearn.utils import resample
from sklearn.model_selection import train_test_split

import re
import random
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pprint
import nltk

import warnings
warnings.filterwarnings('ignore')

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
cd drive/MyDrive/w266/style_transfer_w266/

/content/drive/MyDrive/w266/style_transfer_w266


In [8]:
# train_file = 'drive/MyDrive/data/original-train.tsv'
# dev_file = 'drive/MyDrive/data/original-dev.tsv'
# test_file = 'drive/MyDrive/data/original-test.tsv'
train_file = 'data/original-train.tsv'
dev_file = 'data/original-dev.tsv'
test_file = 'data/original-test.tsv'
df_train = pd.read_csv(train_file, sep='\t')
df_dev = pd.read_csv(dev_file, sep='\t')
df_test = pd.read_csv(test_file, sep='\t')

In [8]:
print(f'''mean length of offensive text: {df_train['offensive-text'].map(len).mean()}''')
print(f'''min length of offensive text: {df_train['offensive-text'].map(len).min()}''')
print(f'''max length of offensive text: {df_train['offensive-text'].map(len).max()}''')
print(f'''mean length of neutralized text: {df_train['style-transferred-text'].map(len).mean()}''')
print(f'''min length of neutralized text: {df_train['style-transferred-text'].map(len).min()}''')
print(f'''max length of neutralized text: {df_train['style-transferred-text'].map(len).max()}''')

mean length of offensive text: 69.85353535353535
min length of offensive text: 9
max length of offensive text: 238
mean length of neutralized text: 60.48800505050505
min length of neutralized text: 1
max length of neutralized text: 174


# Trial with Flan T5

In [9]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

t5tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
t5model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base", device_map="auto", torch_dtype=torch.float16)

In [10]:
#@title 3-Shot Learning
df_fewshot = df_test.copy()
n_shots = 3

for row in range(len(df_fewshot)):
    #For each item in the test set, we use 3 examples from the training set to as n-shots
    sources_targets = df_train.sample(n_shots, replace=False)

    for i in range(n_shots):
        df_fewshot.loc[row, f'shot{i+1}_source'] = sources_targets.iloc[i,0]
        df_fewshot.loc[row, f'shot{i+1}_target'] = sources_targets.iloc[i,1]


df_fewshot = df_fewshot.rename(columns={'offensive-text':'source', 'style-transferred-text':'target'})

In [11]:
df_fewshot['prompt'] = df_fewshot.apply(lambda x: 
                 'Rewrite the toxic text in non-toxic style. \n\n'
                 'Toxic text: ' + x['shot1_source'] + '\n' + 'Non-toxic text: ' + x['shot1_target'] + '\n\n' + \
                 'Toxic text: ' + x['shot2_source'] + '\n' + 'Non-toxic text: ' + x['shot2_target'] + '\n\n' + \
                 'Toxic text: ' + x['shot3_source'] + '\n' + 'Non-toxic text: ' + x['shot3_target'] + '\n\n' + \
                 'Toxic text: ' + x['source'] + '\n' + 'Non-toxic text: ', axis=1)

In [ ]:
print(df_fewshot.prompt[0])

Rewrite the toxic text in non-toxic style. 

Toxic text: Has been a US military puppet from jump
Non-toxic text: Has been a tool for the US military from the beginning

Toxic text: It's not though you fucking idiot
Non-toxic text: It's not though

Toxic text: Maybe you shouldn't turn your new country into the shithole you just left.
Non-toxic text: You should be careful not to bring your old problems from your old country to your new country.

Toxic text: So maybe you should be more retarded.
Non-toxic text: 


In [12]:
for i in range(len(df_fewshot)):
    if i%10 == 0:
        print(f'Working on number: {i}')
    t5_inputs = t5tokenizer([df_fewshot.prompt[i]], return_tensors='pt').input_ids.to("cuda")
    t5_summary_ids = t5model.generate(t5_inputs, 
                                    num_beams=5,
                                    no_repeat_ngram_size=1,
                                    num_return_sequences=1,
                                    top_p = 0.92,
                                    top_k = 100,
                                    min_length = 1,
                                    max_length = 200,
                                    do_sample=True,
                                    )
    output = t5tokenizer.batch_decode(t5_summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    df_fewshot.loc[i,'fewshot_output'] = output

Working on number: 0
Working on number: 10
Working on number: 20
Working on number: 30
Working on number: 40
Working on number: 50
Working on number: 60
Working on number: 70
Working on number: 80
Working on number: 90
Working on number: 100
Working on number: 110
Working on number: 120
Working on number: 130
Working on number: 140
Working on number: 150
Working on number: 160
Working on number: 170
Working on number: 180
Working on number: 190


In [13]:
df_fewshot.loc[:,['source','target','prompt','fewshot_output']]

,source,target,prompt,fewshot_output
0,So maybe you should be more retarded.,So maybe you should be more backward,Rewrite the toxic text in non-toxic style. \n\...,You should be more retarded.
1,THERES A MEGATHREAD FOR VACCINE OR COVID RELAT...,THERES ACTUALLY A MEGATHREAD FOR VACCINE OR CO...,Rewrite the toxic text in non-toxic style. \n\...,THERE IS A MEGATHREAD FOR VACCINE OR COVIDE RE...
2,the fuck.... you are on drugs,uh..... you are not being realistic,Rewrite the toxic text in non-toxic style. \n\...,the fuck.... you are on drugs
3,NTA. Dump his ass. Tablemanners are no rocke...,You should leave him. Tablemanners are no rock...,Rewrite the toxic text in non-toxic style. \n\...,Tablemanners are no rocket science. Treating o...
4,Youre soft as baby shit,Youre really soft,Rewrite the toxic text in non-toxic style. \n\...,You're soft as baby shit.
...,...,...,...,...
194,"NTA, Jes-us your brother is dumb. How long did...","NTA, your brother is not thinking straight. Ho...",Rewrite the toxic text in non-toxic style. \n\...,Jes-us your brother is dumb. How long did he t...
195,Formula one…nascar….shit it exactly the same e...,Formula one…nascar….it exactly the same except...,Rewrite the toxic text in non-toxic style. \n\...,Formula one...nascar....shit it exactly the sa...
196,"BRB, gonna go call the mods pathetic egotistic...","BRB, gonna go call the mods.",Rewrite the toxic text in non-toxic style. \n\...,"BRB, gonna go call the mods stupid neck beard."
197,CUCKOLD Carlson is a problem,Carlson is the problem,Rewrite the toxic text in non-toxic style. \n\...,Carlson is a problem.


In [14]:
# df_fewshot.to_csv('test_output.csv',sep='\t',index=False)
df_fewshot.to_csv('outputs/t5_few_shot_output.csv',sep='\t',index=False)

## Evaluation with ROUGE

In [10]:
import evaluate

rouge = evaluate.load('rouge')
bleu = evaluate.load('bleu')
# bleurt = evaluate.load('bleurt')

In [16]:
#@title Baseline Score on the source and target
print(rouge.compute(predictions=df_fewshot.source,
              references=df_fewshot.target))
print(bleu.compute(predictions=df_fewshot.source,
              references=df_fewshot.target))

# If my predictions did nothing but repeat the same toxic text, I'd get these scores

{'rouge1': 0.6887918451280337, 'rouge2': 0.5605071385408666, 'rougeL': 0.6829326508983988, 'rougeLsum': 0.6845511504130748}
{'bleu': 0.5391232310503405, 'precisions': [0.6839945280437757, 0.5702752293577982, 0.49604117181314333, 0.436613665663945], 'brevity_penalty': 1.0, 'length_ratio': 1.1498230436492332, 'translation_length': 2924, 'reference_length': 2543}


In [17]:
#@title Score after few shot learning
print(rouge.compute(predictions=df_fewshot.fewshot_output,
              references=df_fewshot.target))
print(bleu.compute(predictions=df_fewshot.fewshot_output,
              references=df_fewshot.target))

{'rouge1': 0.6416778124372912, 'rouge2': 0.49360163541682245, 'rougeL': 0.633482217077764, 'rougeLsum': 0.6344573837972729}
{'bleu': 0.446899836804514, 'precisions': [0.6404580152671756, 0.4878149524989674, 0.39693969396939693, 0.32164031620553357], 'brevity_penalty': 1.0, 'length_ratio': 1.0302791977978765, 'translation_length': 2620, 'reference_length': 2543}


## Evaluate with Classification Model - NonToxicScore

In [11]:
import sys
sys.path.append('./notebooks')
from DistilBertClassification import BertClassificationML, NonToxicScoreDataLoader, NonToxicScore

# Load DistilBERT Classification Model to calculate NonToxicScore
score_model = BertClassificationML()
score_model = score_model.to(device)

# Load training weights
pretrained_weights = torch.load('models/DistilBertToxicClassification5.pth')
score_model.load_state_dict(pretrained_weights )

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [14]:
output_file = 'outputs/t5_few_shot_output.csv'
output_col = 'fewshot_output'

# Create Data Loader
score_loader = NonToxicScoreDataLoader(output_file, output_col)

# Calculate NonToxicScore
fewshot_NonToxicScores, avg_score = NonToxicScore(score_loader, score_model)

{'NonToxicScore': 0.21234116579017523}


# Tune the model with training set, then do few-shot learning again

## Normal Training (no Trainer)

In [12]:
from datasets import load_metric, load_dataset

In [15]:
class ToxicData(Dataset):
    def __init__(self, data, tokenizer):
        super(Dataset, self).__init__()
        self.source_texts = 'transfer to nontoxic text: ' + data['offensive-text']
        self.target_texts = data['style-transferred-text']
        self.tokenizer=tokenizer

    def __len__(self):
        return len(self.target_texts)

    def __getitem__(self, idx):
        source_text = self.source_texts[idx]
        target_text = self.target_texts[idx]

        source_tokens = self.tokenizer.encode(source_text, padding='max_length',truncation=True)
        source_tensor = torch.tensor(source_tokens, dtype=torch.long)

        target_tokens = self.tokenizer.encode(target_text, padding='max_length',truncation=True)
        target_tensor = torch.tensor(target_tokens, dtype=torch.long)

        # # important: we need to replace the index of the padding tokens by -100
        # # such that they are not taken into account by the CrossEntropyLoss
        # labels_with_ignore_index = []
        # for labels_example in target_tokens.input_ids:
        #   labels_example = [label if label != 0 else -100 for label in labels_example]
        #   labels_with_ignore_index.append(labels_example)
        
        # source_tokens["labels"] = labels_with_ignore_index

        if torch.cuda.is_available():
            source_tensor = source_tensor.cuda()
            target_tensor = target_tensor.cuda()

        return source_tensor, target_tensor

In [16]:
train_file = 'data/original-train.tsv'
dev_file = 'data/original-dev.tsv'
test_file = 'data/original-test.tsv'

In [17]:
dataset = load_dataset('csv', sep="\t", data_files={'train': train_file, 'validation': dev_file,'test': test_file})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d836ef9b772838ed/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [33]:
train_ds = dataset['train']
val_ds = dataset['validation']
test_ds = dataset['test']

In [32]:
train_ds

Dataset({
    features: ['offensive-text', 'style-transferred-text'],
    num_rows: 1584
})

In [34]:
example = train_ds[0]
org = example['offensive-text']
mod = example['style-transferred-text']
print("Original Text:", org)
print("Style Transfered Text:", mod)
     

Original Text: Pussy nobody asked for your input.
Style Transfered Text: Nobody asked for your input


In [50]:
prefix = 'transfer to nontoxic text: '
max_input_length = 64
max_target_length = 64

def preprocess_examples(examples):
  # encode the documents
  orgs = examples['offensive-text']
  mods = examples['style-transferred-text']
  
  inputs = [prefix + org for org in orgs]
  model_inputs = t5tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

  # encode the summaries
  labels = t5tokenizer(mods, max_length=max_target_length, padding="max_length", truncation=True).input_ids

  # important: we need to replace the index of the padding tokens by -100
  # such that they are not taken into account by the CrossEntropyLoss
  labels_with_ignore_index = []
  for labels_example in labels:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)
  
  model_inputs["labels"] = labels_with_ignore_index

  return model_inputs

In [51]:
encoded_train_ds = train_ds.map(preprocess_examples, batched=True, remove_columns=train_ds.column_names)
encoded_val_ds = val_ds.map(preprocess_examples, batched=True, remove_columns=val_ds.column_names)
encoded_test_ds = test_ds.map(preprocess_examples, batched=True, remove_columns=test_ds.column_names)

In [56]:
t5tokenizer.decode(encoded_train_ds[0]['input_ids'])

'transfer to nontoxic text: Pussy nobody asked for your input.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [57]:
labels = encoded_train_ds[0]['labels']
print(labels)

[22009, 1380, 21, 39, 3785, 1, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]


In [58]:
t5tokenizer.decode([x for x in labels if x != -100])

'Nobody asked for your input</s>'

In [59]:
encoded_train_ds.set_format(type="torch")
encoded_val_ds.set_format(type="torch")
encoded_test_ds.set_format(type="torch")

In [60]:
def create_dataloaders(train_batch_size=8, eval_batch_size=32):
    train_dataloader = DataLoader(encoded_train_ds, shuffle=True, batch_size=train_batch_size)
    val_dataloader = DataLoader(encoded_val_ds, shuffle=False, batch_size=eval_batch_size)
    
    return train_dataloader, val_dataloader

In [72]:
test_dataloader = DataLoader(encoded_test_ds, shuffle=False, batch_size=8)

In [66]:
hyperparameters = {
    "learning_rate": 2e-5,
    "num_epochs": 4, # set to very high number
    "train_batch_size": 8, # Actual batch size will this x 8 
    "eval_batch_size": 8, # Actual batch size will this x 8 
    "seed": 42,
    "patience": 3, # early stopping
    "output_dir": "/models/",
}

In [67]:
from transformers import T5ForConditionalGeneration, AdamW, set_seed
from accelerate import Accelerator
from tqdm.notebook import tqdm
import datasets
import transformers

def training_function():
    # Initialize accelerator
    accelerator = Accelerator()

    # To have only one message (and not 8) per logs of Transformers or Datasets, we set the logging verbosity
    # to INFO for the main process only.
    if accelerator.is_main_process:
        datasets.utils.logging.set_verbosity_warning()
        transformers.utils.logging.set_verbosity_info()
    else:
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()

    # The seed need to be set before we instantiate the model, as it will determine the random head.
    set_seed(hyperparameters["seed"])

    # Instantiate the model, let Accelerate handle the device placement.
    model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
    model = model.to(device)

    # Instantiate optimizer
    optimizer = AdamW(model.parameters(), lr=hyperparameters["learning_rate"])

    # Prepare everything
    train_dataloader, val_dataloader = create_dataloaders(
        train_batch_size=hyperparameters["train_batch_size"], eval_batch_size=hyperparameters["eval_batch_size"]
    )
    # unpack the objects in the same order we gave them to the prepare method.
    model, optimizer, train_dataloader, val_dataloader = accelerator.prepare(model, optimizer, 
                                                                             train_dataloader, val_dataloader)
    # train the model
    epochs_no_improve = 0
    min_val_loss = 1000000
    
    for epoch in range(hyperparameters["num_epochs"]):
        # We only enable the progress bar on the main process to avoid having 8 progress bars.
        progress_bar = tqdm(range(len(train_dataloader)), disable=not accelerator.is_main_process)
        progress_bar.set_description(f"Epoch: {epoch}")
        model.train()
        for batch in train_dataloader:
            outputs = model(**batch)
            loss = outputs.loss
            accelerator.backward(loss)
            
            optimizer.step()
            optimizer.zero_grad()
            progress_bar.set_postfix({'loss': loss.item()})
            progress_bar.update(1)

        # Evaluate at the end of the epoch (distributed evaluation as we have 8 TPU cores)
        model.eval()
        validation_losses = []
        for batch in val_dataloader:
            with torch.no_grad():
                outputs = model(**batch)
            loss = outputs.loss

            # We gather the loss from the 8 TPU cores to have them all.
            validation_losses.append(accelerator.gather(loss[None]))

        # Compute average validation loss
        val_loss = torch.stack(validation_losses).sum().item() / len(validation_losses)
        # Use accelerator.print to print only on the main process.
        accelerator.print(f"epoch {epoch}: validation loss:", val_loss)
        if val_loss < min_val_loss:
          epochs_no_improve = 0
          min_val_loss = val_loss

    return model

In [68]:
model = training_function()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/c782cba52f8ea6a704240578055cf1c3fc2f2ca9/config.json
Model config T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num

  0%|          | 0/198 [00:00<?, ?it/s]

epoch 0: validation loss: 1.0558067321777345


  0%|          | 0/198 [00:00<?, ?it/s]

epoch 1: validation loss: 1.021386947631836


  0%|          | 0/198 [00:00<?, ?it/s]

epoch 2: validation loss: 1.0070569610595703


  0%|          | 0/198 [00:00<?, ?it/s]

epoch 3: validation loss: 0.9924876403808593


In [69]:
# from accelerate import notebook_launcher
# notebook_launcher(training_function)

In [70]:
# save training weights
torch.save(model.state_dict(), 'models/t5-detoxify-base1.pth')

In [1]:
# t5_inputs = t5tokenizer([df_fewshot.prompt[i]], return_tensors='pt').input_ids.to("cuda")
# t5_inputs = encoded_test_ds['input_ids'].to(device)
# t5_test_output_ids = model.generate(t5_inputs, 
#                                     do_sample=True, 
#                                     max_length=50, 
#                                     top_k=0, 
#                                     temperature=0.7
#                                     )


In [ ]:
output = t5tokenizer.batch_decode(t5_summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

In [ ]:
# Inference
def T5Detoxify(data_loader, model, verbose=True):
    # Initialize
    outputs=[]

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        for step, data in enumerate(data_loader):
            
            # send the data to cuda device
            ids = data[0].to(device, dtype = torch.long)
            mask = data[1].to(device, dtype = torch.long)

            # compute output / NonToxicScore
            output = model(input_ids=ids,
                           attention_mask=mask)

            outputs.extend(output.cpu().detach().numpy().tolist())
      

    avg_NonToxicScore = np.mean(outputs)
    metrics = {"NonToxicScore": avg_NonToxicScore}
    # elapsed_time = time.time() - t0
    if verbose:
      print(metrics)
    
    return outputs, metrics

## Train T5 with Trainer

In [36]:
prefix = 'transfer to nontoxic text: '
max_input_length = 64
max_target_length = 64

def clean_text(text):
  sentences = nltk.sent_tokenize(text.strip())
  sentences_cleaned = [s for sent in sentences for s in sent.split("\n")]
  # sentences_cleaned_no_titles = [sent for sent in sentences_cleaned
  #                                if len(sent) > 0 and
  #                                sent[-1] in string.punctuation]
  # text_cleaned = "\n".join(sentences_cleaned_no_titles)
  text_cleaned = "\n".join(sentences_cleaned)
  return text_cleaned

def preprocess_data(examples):
#   texts_cleaned = [clean_text(text) for text in examples["text"]]
  source_inputs = [prefix + text for text in examples['offensive-text']]
  model_inputs = t5tokenizer(source_inputs, max_length=max_input_length, padding="max_length", truncation=True)
  
  target_inputs = [text for text in examples['style-transferred-text']]
  target_tokens = t5tokenizer(target_inputs, max_length=max_target_length, padding="max_length", truncation=True)
  
  # important: we need to replace the index of the padding tokens by -100
  # such that they are not taken into account by the CrossEntropyLoss
  labels_with_ignore_index = []
  for labels_example in target_tokens.input_ids:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)
  
  model_inputs["labels"] = labels_with_ignore_index
  # model_inputs["labels"] = target_tokens
  
  return model_inputs

In [44]:
tokenized_datasets = dataset.map(preprocess_data, batched=True)
encoded_train_ds = dataset['train'].map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)
encoded_val_ds = dataset['validation'].map(preprocess_data, batched=True, remove_columns=dataset['validation'].column_names)
encoded_test_ds = dataset['test'].map(preprocess_data, batched=True, remove_columns=dataset['test'].column_names)

Map:   0%|          | 0/1584 [00:00<?, ? examples/s]

Map:   0%|          | 0/198 [00:00<?, ? examples/s]

Map:   0%|          | 0/199 [00:00<?, ? examples/s]

Map:   0%|          | 0/1584 [00:00<?, ? examples/s]

Map:   0%|          | 0/198 [00:00<?, ? examples/s]

Map:   0%|          | 0/199 [00:00<?, ? examples/s]

In [46]:
# preprocess_data(dataset['train'][0])
# tokenized_datasets['train']['input_ids'][0][0]
t5tokenizer.decode(tokenized_datasets['train'][0]['input_ids'])

'transfer to nontoxic text: Pussy nobody asked for your input.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [45]:
t5tokenizer.decode(encoded_train_ds[0]['input_ids'])

'transfer to nontoxic text: Pussy nobody asked for your input.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [47]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['offensive-text', 'style-transferred-text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1584
    })
    validation: Dataset({
        features: ['offensive-text', 'style-transferred-text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 198
    })
    test: Dataset({
        features: ['offensive-text', 'style-transferred-text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 199
    })
})

In [49]:
encoded_train_ds

In [71]:
batch_size = 8
model_name = "google/flan-t5-base-tuned-toxicity"
model_dir = f"models/{model_name}"

args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    # evaluation_strategy="steps",
    # eval_steps=100,
    # logging_strategy="steps",
    # logging_steps=100,
    # save_strategy="steps",
    # save_steps=200,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    # save_total_limit=3,
    num_train_epochs=7,
    predict_with_generate=True,
    # fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    # report_to="tensorboard",
    remove_unused_columns=False
)

In [53]:
data_collator = DataCollatorForSeq2Seq(t5tokenizer)
metric = load_metric("rouge")

In [54]:
import nltk

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = t5tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, t5tokenizer.pad_token_id)
    decoded_labels = t5tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) 
                      for label in decoded_labels]
    
    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != t5tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [72]:
from transformers import AdamW, get_cosine_schedule_with_warmup

t5model = T5ForConditionalGeneration.from_pretrained(model_ckpt)
t5model = t5model.to(device)

optimizer = AdamW(t5model.parameters(), lr=2e-5)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=10, num_training_steps=1000)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

In [73]:
model_ckpt = "google/flan-t5-base"
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_ckpt, return_dict=True)

trainer = Seq2SeqTrainer(
    # model_init=model_init,
    model=t5model,
    args=args,
    # train_dataset=tokenized_datasets['train'],
    train_dataset=encoded_train_ds, 
    # eval_dataset=tokenized_datasets['validation'],
    eval_dataset=encoded_val_ds,
    data_collator=data_collator,
    tokenizer=t5tokenizer,
    optimizers=(optimizer, scheduler),
    compute_metrics=compute_metrics
)

In [43]:
# %load_ext tensorboard
# %tensorboard --logdir '{model_dir}'/runs

In [74]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.271900,1.075335,66.937800,53.924800,66.826400,66.839700,16.262600
2,1.105600,1.038640,67.771300,54.549400,67.534200,67.488700,15.994900
3,1.038300,1.020302,68.550700,55.503900,68.421300,68.402700,15.868700
4,1.009200,1.017749,68.340200,55.432200,68.186700,68.147300,15.833300
5,1.002400,1.016863,68.070700,55.278900,67.873900,67.951600,15.848500
6,0.989500,1.017735,68.233600,55.362800,68.034400,68.118500,15.823200
7,0.984800,1.013406,68.084900,55.497200,67.926500,67.993100,15.818200


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.271900,1.075335,66.937800,53.924800,66.826400,66.839700,16.262600
2,1.105600,1.038640,67.771300,54.549400,67.534200,67.488700,15.994900
3,1.038300,1.020302,68.550700,55.503900,68.421300,68.402700,15.868700
4,1.009200,1.017749,68.340200,55.432200,68.186700,68.147300,15.833300
5,1.002400,1.016863,68.070700,55.278900,67.873900,67.951600,15.848500
6,0.989500,1.017735,68.233600,55.362800,68.034400,68.118500,15.823200
7,0.984800,1.013406,68.084900,55.497200,67.926500,67.993100,15.818200


TrainOutput(global_step=1386, training_loss=1.057384100139227, metrics={'train_runtime': 930.1726, 'train_samples_per_second': 11.92, 'train_steps_per_second': 1.49, 'total_flos': 949073690492928.0, 'train_loss': 1.057384100139227, 'epoch': 7.0})

In [75]:
trainer.evaluate()

{'eval_loss': 1.0203020572662354,
 'eval_rouge1': 68.5507,
 'eval_rouge2': 55.5039,
 'eval_rougeL': 68.4213,
 'eval_rougeLsum': 68.4027,
 'eval_gen_len': 15.8687,
 'eval_runtime': 38.25,
 'eval_samples_per_second': 5.176,
 'eval_steps_per_second': 0.654,
 'epoch': 7.0}

In [76]:
trainer.evaluate(encoded_test_ds)

{'eval_loss': 0.9976702332496643,
 'eval_rouge1': 66.7551,
 'eval_rouge2': 54.7416,
 'eval_rougeL': 66.3511,
 'eval_rougeLsum': 66.4788,
 'eval_gen_len': 15.2613,
 'eval_runtime': 25.156,
 'eval_samples_per_second': 7.911,
 'eval_steps_per_second': 0.994,
 'epoch': 7.0}

In [77]:
# save training weights
trainer.save_model('models/t5-detoxify-base2.pth')
# torch.save(model.state_dict(), 'models/t5-detoxify-base1.pth')

In [78]:
t5model.eval()

test_dataloader = DataLoader(encoded_test_ds, batch_size=8, collate_fn=data_collator)

generated_texts = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        output = t5model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=64)
        generated_texts.extend(t5tokenizer.batch_decode(output, skip_special_tokens=True))



In [80]:
generated_texts[:10]

['So maybe you should be more retarded.',
 "THERE IS A MEGATHREAD FOR VACCINE OR COVID RELATED TOPICS. DON'T TALK ABOUT THAT HERE",
 'you are on drugs',
 'NTA. Dump his ass. Tablemanners are no rocket science. Treating other people like human beings is no rocket science. His still a child.',
 "You're not very smart",
 'Because everyone knows why. It’s Jadas fault. Will was just triggered because of all the things he has to deal with because of her.',
 'And “poor kids are just as bright as white kids”.... Your blundering bafoon chooses racism as well',
 "Tucker is a racist person that no one will miss when he's gone... he's also racist",
 "There's plenty of evidence, it's exactly why they won't allow forensic audits *wipes voting machines clean of their records*",
 'Came here to say this, Carlsen is a freaking moron and defines recreational outrage imo']

In [84]:
# Create a new DataFrame with the generated texts
df_gen = pd.DataFrame({'t5_tuning_output': generated_texts})

# Add the generated_text column to the df_test dataframe
df_t5_output = pd.concat([df_test, df_gen], axis=1)

df_t5_output.head(10)

,offensive-text,style-transferred-text,t5_tuning_output
0,So maybe you should be more retarded.,So maybe you should be more backward,So maybe you should be more retarded.
1,THERES A MEGATHREAD FOR VACCINE OR COVID RELAT...,THERES ACTUALLY A MEGATHREAD FOR VACCINE OR CO...,THERE IS A MEGATHREAD FOR VACCINE OR COVID REL...
2,the fuck.... you are on drugs,uh..... you are not being realistic,you are on drugs
3,NTA. Dump his ass. Tablemanners are no rocke...,You should leave him. Tablemanners are no rock...,NTA. Dump his ass. Tablemanners are no rocket ...
4,Youre soft as baby shit,Youre really soft,You're not very smart
5,Because everyone knows why. It’s jadas fault. ...,Because everyone knows it's jadas fault. Will ...,Because everyone knows why. It’s Jadas fault. ...
6,And “poor kids are just as bright as white kid...,"He said ""poor kids are just as bright as white...",And “poor kids are just as bright as white kid...
7,Tucker is a garbage person that no one will mi...,Tucker is a bad person who instigates racism,Tucker is a racist person that no one will mis...
8,"There’s plenty of evidence, it’s exactly why t...","There is evidence, which is why they won't all...","There's plenty of evidence, it's exactly why t..."
9,"Came here to say this, carlsen is a freaking m...","Came here to say this, carlsen is pretty unint...","Came here to say this, Carlsen is a freaking m..."


In [85]:
# Save the updated dataframe as a CSV file
df_t5_output.to_csv('outputs/t5_tuning_output.csv',sep='\t',index=False)

In [87]:
## Calculate NonToxicScore
output_file = 'outputs/t5_tuning_output.csv'
output_col = 't5_tuning_output'

# Create Data Loader
score_loader = NonToxicScoreDataLoader(output_file, output_col)

# Calculate NonToxicScore
fewshot_NonToxicScores, avg_score = NonToxicScore(score_loader, score_model)

{'NonToxicScore': 0.3005175284559291}
